## Linear Regression

In [19]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

from helpers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the data

In [20]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci')
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


## Linear Regression

In [4]:
lr = linear_model.LinearRegression()

X_train = train_input.view(train_input.size(0), -1)
X_test = test_input.view(test_input.size(0), -1)

# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
lr.fit(X=X_train, y=train_target)
preds = lr.predict(X_test)

preds = torch.FloatTensor(preds)

In [5]:
preds = discrete_predictions(preds)
accuracy = compute_accuracy(test_target, preds)
print('Accuracy ' + str(accuracy*100) + '%')

Accuracy 73.0%


## Neural Network

In [40]:
# load data
train_input , train_target = bci.load(root = './data_bci')
test_input , test_target = bci.load(root = './data_bci', train = False)

In [41]:
train_input2 = train_input[0:300]
train_target2 = train_target[0:300]

In [32]:
# normalize mean to 0
train_input.sub_(train_input.mean())
test_input.sub_(test_input.mean())

# normalize variance to 1
train_input.div_(train_input.std())
test_input.div_(test_input.std())

print('done')

done


In [42]:
class Net(nn.Module):
    def __init__(self, nb_hidden=64):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(640, 2)

    def forward(self, x):
        #print("======")
        #print(x.size())
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        #print(x.size())
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        #print(x.size())
        x = F.tanh(self.fc1(x.view(-1, 640)))
        #print(x.size())
        #print("======")
        return x

In [59]:
my_cool_net = Net()
train_model(my_cool_net, Variable(train_input2.view(-1, 1, 28, 50)), Variable(train_target2), 10, nb_epochs=50, step_size=1e-2)

0: 21.703055202960968
1: 20.777284920215607
2: 20.587969839572906
3: 20.359611749649048
4: 20.005020201206207
5: 19.586762249469757
6: 19.022407293319702
7: 18.390847593545914
8: 17.583850502967834
9: 16.90441805124283
10: 16.22645789384842
11: 15.550996750593185
12: 14.910525143146515
13: 14.232996761798859
14: 14.136222004890442
15: 13.863037198781967
16: 12.768241316080093
17: 11.960491716861725
18: 11.502115219831467
19: 11.319451689720154
20: 10.929494619369507
21: 9.91102509200573
22: 9.63757948577404
23: 9.272901833057404
24: 9.108431920409203
25: 8.391026750206947
26: 7.939201042056084
27: 7.767505526542664
28: 7.292846530675888
29: 7.086508825421333
30: 6.755961567163467
31: 6.664888724684715
32: 6.392997086048126
33: 6.45441997051239
34: 6.230731621384621
35: 5.868110284209251
36: 5.824722006917
37: 5.683975234627724
38: 5.499234184622765
39: 5.4962416142225266
40: 5.536109626293182
41: 5.207741603255272
42: 5.139477074146271
43: 5.0484001487493515
44: 4.982447117567062
45: 4

#### Compute error rate

In [60]:
nb_errors = compute_nb_errors(my_cool_net, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)

In [61]:
nb_errors/test_input.size(0)

0.38

#### Grid search on model parameters

In [ ]:
step_sizes = [0.01, 0.005, 0.0025, 0.001, 0.0005]
nb_iters = [25, 50, 100, 250, 500]

res = []

for lr in step_sizes:
    results = []